In [18]:
import numpy as np

def rotary_positional_embedding(x: np.ndarray, 
                              seq_len: int, 
                              embed_dim: int):
    
    """
    Áp dụng Rotary Positional Embedding cho vector ẩn.
    
    Args:
        x: Tensor input (batch_size, seq_len, dim).
        seq_len: Chiều dài chuỗi (sequence length).
        dim: Kích thước vector (embedding dimension).
        
    Returns:
        Tensor sau khi áp dụng RoPE.
    """

    # 1. Tách vector thành (even, odd)
    x_even = x[:, :, 0::2]  # Các giá trị tại chỉ số chẵn
    x_odd = x[:, :, 1::2]   # Các giá trị tại chỉ số lẻ


    # 2. Tính góc xoay
    positions = np.arange(start=0, stop=seq_len)[:, None] # # Vị trí từng token, shape: [seq_len, 1]
    theta = np.arange(0, embed_dim, 2) / embed_dim # 2* k / d. Shape: [embed_dim / 2, ]
    theta = 1 / (10000 ** (theta)) # 1 / (10000 ** (2 *k / d)). Shape: [embed_dim / 2, ]
    # print("positions: \n", positions)
    # print("theta: \n", theta)
    angles = positions * theta # theta
    # print("angles: \n", angles)

    # 3. Tính cos/sin của góc
    cos_angles = np.cos(angles)
    sin_angles = np.sin(angles)

    # 4. Áp dụng phép xoay cho từng cặp
    x_rotated_even = x_even * cos_angles - x_odd * sin_angles
    x_rotated_odd = x_even * sin_angles + x_odd * cos_angles

    # ghép lại
    x_rotated = np.ones_like(x)
    x_rotated[:, :, 0::2] = x_rotated_even
    x_rotated[:, :, 1::2] = x_rotated_odd

    return x_rotated

N = 2
seq_len = 5
embed_dim = 4

mock_data = np.random.randn(N, seq_len, embed_dim)

# In dữ liệu gốc
print("Input ban đầu: \n", mock_data)

# Áp dụng RoPE
x_rotated = rotary_positional_embedding(x=mock_data, seq_len=seq_len, embed_dim=embed_dim)

# In kết quả
print("\nKết quả sau khi áp dụng RoPE:\n",  x_rotated)


Input ban đầu: 
 [[[-0.24314441  0.64129783 -1.48134646  0.20391021]
  [ 1.64537239 -1.19141783  0.26323719  0.75622655]
  [-0.05689482 -0.57656691 -0.90053521 -0.55618454]
  [-0.39824614  0.47397978  0.0374363   0.16878115]
  [ 2.50906045 -0.40204966  0.13305711  0.44976982]]

 [[-0.68544005 -0.11787256  0.27565739 -0.18706384]
  [-1.82798017  0.54962275 -0.50693675 -2.09208557]
  [-1.83634144  1.00950172 -1.09028858  0.74820996]
  [-0.91144391 -1.01720723 -0.07255097  0.4692317 ]
  [-0.16313392 -1.07272326  1.46489373 -0.7712628 ]]]

Kết quả sau khi áp dụng RoPE:
 [[[-0.24314441  0.64129783 -1.48134646  0.20391021]
  [ 1.89154203  0.74080732  0.25566189  0.75882106]
  [ 0.54794741  0.18820218 -0.88923216 -0.57408281]
  [ 0.32737266 -0.52543692  0.03235678  0.16982812]
  [-1.94430355 -1.63606602  0.11496468  0.45473091]]

 [[-0.68544005 -0.11787256  0.27565739 -0.18706384]
  [-1.4501535  -1.24122983 -0.48599089 -2.09705024]
  [-0.15374964 -2.08988149 -1.10503373  0.72625601]
  [ 1.045